# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Лабораторная работа №3

### JSON

In [79]:
import json
import pickle

with open("addres-book.json", "r", encoding="utf-8") as f:
    data = json.load(f)

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [80]:
class User:

    def __init__(self, username, name, sex, address, mail, jobs, id):
        self.username = username
        self.name = name
        self.sex = sex
        self.address = address
        self.mail = mail
        self.jobs = jobs
        self.id = id

    def __str__(self) -> str:
        jobs_str = ", ".join(self.jobs)
        return f"\nId: {self.id}\nUsername: {self.username}\nName: {self.name}\nSex: {self.sex}\naddress: {self.address}\nmail: {self.mail}\njobs: {jobs_str}\n\n#########################"


with open("contributors_sample.json", "r", encoding="utf8") as f:
    data = json.load(f)

user_l = []
for i in data:
    user_l.append(User(**i))

for usr in user_l[:3]:
    print(usr)


Id: 35193
Username: uhebert
Name: Lindsey Nguyen
Sex: F
address: 01261 Cameron Spring
Taylorfurt, AK 97791
mail: jsalazar@gmail.com
jobs: Energy engineer, Engineer, site, Environmental health practitioner, Biomedical scientist, Jewellery designer

#########################

Id: 91970
Username: vickitaylor
Name: Cheryl Lewis
Sex: F
address: 66992 Welch Brooks
Marshallshire, ID 56004
mail: bhudson@gmail.com
jobs: Music therapist, Volunteer coordinator, Designer, interior/spatial

#########################

Id: 1848091
Username: sheilaadams
Name: Julia Allen
Sex: F
address: Unit 1632 Box 2971
DPO AE 23297
mail: darren44@yahoo.com
jobs: Management consultant, Engineer, structural, Lecturer, higher education, Theatre manager, Designer, textile

#########################


1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [81]:
l = []
for i in data:
    dom = i["mail"].split("@")[1]
    if dom not in l:
        l.append(i["mail"].split("@")[1])

print(l)


['gmail.com', 'yahoo.com', 'hotmail.com']


1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [82]:
# введём настоящий юзернейм
def find_username(username):
    try:
        for item in data:
            if username == item['username']:
                print(item)
                return

        raise ValueError
    except ValueError:
        print('Error: user not found')

find_username(input())

uhebert
{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}


In [39]:
# выведем ошибку
def find_username(username):
    try:
        for item in data:
            if username == item['username']:
                print(item)
                return

        raise ValueError
    except ValueError:
        print('Error: user not found')

find_username(input())

daria147
Error: user not found


1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [83]:
def male_female():
    m, f = 0, 0
    for i in data:
        if i["sex"] == "M":
            m += 1
        else:
            f += 1
    return f"Мужчин: {m}, женщин: {f}"

male_female()

'Мужчин: 2064, женщин: 2136'

1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [42]:
import pandas as pd
    
contributors = pd.read_json("contributors_sample.json").drop(["name", "address", "mail", "jobs"], axis=1)
col = contributors.pop("id")
contributors.insert(0, col.name, col)
print(contributors)

           id       username sex
0       35193        uhebert   F
1       91970    vickitaylor   F
2     1848091    sheilaadams   F
3       50969       nicole82   F
4      676820         jean67   M
...       ...            ...  ..
4195   423555  stevenspencer   F
4196    35251      rwilliams   M
4197   135887      lmartinez   F
4198   212714     brendahill   M
4199   344321       mistyray   F

[4200 rows x 3 columns]


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [43]:
recipes = pd.read_csv("recipes_sample.csv", sep=",")
recipes = recipes.merge(contributors, left_on="contributor_id", right_on="id", how="left")
recipes = recipes[pd.isna(recipes["id_y"])] # если информация отсутствует
print(len(recipes["contributor_id"].unique()))

4204


### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [50]:
with open("contributors_sample.json", "r", encoding="utf8") as f:
    data = json.load(f)

d = {}
for i in data:
    for job in i["jobs"]:
        if job not in d:
            d[job] = []
        d[job].append(i["username"])


# print({k: d[k] for k in list(d)})
#выведет весь словарь, для лучшей читаемости выведу лишь часть, чтобы вам не пришлось долго скроллить на гитхабе :) 

print({k: d_pickle[k] for k in list(d_pickle)[:3]})

{'Energy engineer': ['uhebert', 'annmoore', 'garysilva', 'martinezashley', 'sextonsheila', 'pjames', 'smithjonathan', 'wardjames', 'cwheeler', 'ucarlson', 'robert71', 'johnsontheresa', 'amanda41', 'stacey47', 'timothynelson', 'timothynelson', 'rogersmichael', 'melissa94', 'wmcdaniel', 'charles74', 'smithjennifer', 'clintonjones'], 'Engineer, site': ['uhebert', 'nancy12', 'andrea03', 'catherineross', 'wesley32', 'natalieross', 'rossdoris', 'christophersmith', 'dbooker', 'ericarobertson', 'trantricia', 'tpugh', 'jasonvelez', 'samantha36', 'brandidaniels', 'tenglish', 'reyesbrett', 'austin18', 'vjohnson', 'zmejia', 'daniel04', 'cynthia20', 'morgan15', 'avaldez', 'jessica92', 'laurieholloway', 'baileyvictoria'], 'Environmental health practitioner': ['uhebert', 'jonathanchristian', 'xjohnson', 'dsmith', 'james01', 'nancytaylor', 'ztaylor', 'andrewwoods', 'susan54', 'fmaldonado', 'james74', 'bakerjacob', 'stephanie81', 'whitejoseph', 'qolson', 'hknox', 'gonzalesdaniel', 'tranronald', 'jesseg

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [84]:
with open("job_people.pickle", "wb") as f:
    pickle.dump(d, f)

with open("job_people.json", "w") as f:
    json.dump(d, f, indent=1)
    
# объёмы
import os
print("pickle :", os.path.getsize("job_people.pickle"))
print("json   :", os.path.getsize("job_people.json"))

pickle : 132272
json   : 285159


2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [51]:
with open("job_people.pickle", "rb") as f:
    d_pickle = pickle.load(f)

print({k: d_pickle[k] for k in list(d_pickle) [:3]}) 

# выводится тот же словарь

{'Energy engineer': ['uhebert', 'annmoore', 'garysilva', 'martinezashley', 'sextonsheila', 'pjames', 'smithjonathan', 'wardjames', 'cwheeler', 'ucarlson', 'robert71', 'johnsontheresa', 'amanda41', 'stacey47', 'timothynelson', 'timothynelson', 'rogersmichael', 'melissa94', 'wmcdaniel', 'charles74', 'smithjennifer', 'clintonjones'], 'Engineer, site': ['uhebert', 'nancy12', 'andrea03', 'catherineross', 'wesley32', 'natalieross', 'rossdoris', 'christophersmith', 'dbooker', 'ericarobertson', 'trantricia', 'tpugh', 'jasonvelez', 'samantha36', 'brandidaniels', 'tenglish', 'reyesbrett', 'austin18', 'vjohnson', 'zmejia', 'daniel04', 'cynthia20', 'morgan15', 'avaldez', 'jessica92', 'laurieholloway', 'baileyvictoria'], 'Environmental health practitioner': ['uhebert', 'jonathanchristian', 'xjohnson', 'dsmith', 'james01', 'nancytaylor', 'ztaylor', 'andrewwoods', 'susan54', 'fmaldonado', 'james74', 'bakerjacob', 'stephanie81', 'whitejoseph', 'qolson', 'hknox', 'gonzalesdaniel', 'tranronald', 'jesseg

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [57]:
from bs4 import BeautifulSoup

with open('steps_sample.xml') as f:
    data = BeautifulSoup(f, features="xml")
    
recipe_book = {}
for recipe in data.find_all('recipe'):
    id = recipe.find('id')
    if id.text not in recipe_book:
        recipe_book[id.text] = []
    for step in recipe.find_all('step'):
        recipe_book[id.text].append(step.text)

with open('steps_sample.json', 'w') as f:
    json.dump(recipe_book, f) 

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [60]:
step_recipe = {}
for recipe in data_xml.find_all("recipe"):
    step_count = len(recipe.find_all("step"))
    if step_count not in step_recipe:
        step_recipe[step_count] = []
    step_recipe[step_count].append(recipe.find("id").text)

print({k: step_recipe[k] for k in list(step_recipe)[:2]}) # [:2] для лучшей читаемости

{11: ['44123', '302399', '375376', '140610', '374703', '111198', '257111', '432661', '114204', '63069', '165096', '33947', '250024', '330512', '315233', '25259', '331174', '407621', '263019', '112853', '383729', '13709', '336166', '143286', '387284', '290003', '370746', '34833', '11975', '426211', '373582', '88845', '456968', '14149', '507927', '73602', '91981', '175109', '390933', '193208', '83893', '243008', '259789', '303926', '410920', '446605', '32571', '74419', '308056', '78497', '111963', '361181', '302640', '356655', '53743', '57771', '420689', '74520', '50851', '176277', '266814', '27897', '189207', '138771', '279797', '177831', '32515', '256842', '95295', '383349', '109791', '332641', '116993', '173126', '187872', '177681', '249006', '314834', '283033', '117084', '49202', '284916', '247657', '313162', '424727', '227557', '431305', '263038', '439979', '443041', '241042', '258779', '66965', '200503', '109597', '503121', '290595', '401175', '169146', '282228', '316435', '248582'

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [64]:
recipes_with_time = []

for recipe in data_xml.find_all("recipe"):
    for step in recipe.find_all("step"):
        
        if "has_minutes" in step.attrs or "has_hours" in step.attrs:
            recipes_with_time.append(recipe.find("id").text)
            break


print(recipes_with_time_info[:5])

['44123', '35173', '453467', '306168', '50662']


3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [76]:
import numpy as np

recipes = pd.read_csv('recipes_sample.csv', sep=',')
for recipe in data.find_all('recipe'):
    id = int(recipe.find('id').text)
    if np.isnan(recipes[recipes['id'] == id]['n_steps'].values[0]):
        recipes.loc[recipes.id == id, 'n_steps'] = len(recipe.find_all('step'))
recipes        

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,11.0,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3.0,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5.0,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,7.0,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,22.0,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,10.0,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,7.0,this is a delicious soup that i originally fou...,NaN


3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [ ]:
import numpy as np
if recipes["n_steps"].isna().sum() == 0:
    recipes["n_steps"] = recipes["n_steps"].astype(np.int)
    recipes.to_csv("./data/recipes_sample_with_filled_nsteps.csv", index=False)